# CSE 255 HW5

# Jian Li
# A53219654

# New Mexico Area Weather Analysis

## 0. Introduction

This is a report on the historical analysis of weather patterns in an area that approximately overlaps south area of the state of New Mexico. As shown in the map below: the radius of each red circle represents the counts number of records, and the rectangle represents the area being monitored. 
<p><img alt="TMINTMAX.png" src="own_figures/map.png" style="height:280px" /></p>

<p>&nbsp;</p>

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

We focused on six measurements:
* **TMIN:** The daily minimum temperature.
* **TMAX:** The daily maximum temperature.
* **TOBS:** The average temperature for each day.
* **PRCP:** Daily Percipitation (in mm)
* **SNOW:** Daily snowfall (in mm)
* **SNWD:** The depth of accumulated snow.

## 1. Sanity-check: comparison with outside sources

<p>We start by comparing some of the general statistics with graphs that we obtained from <a href="https://en.wikipedia.org/wiki/Silver_City,_New_Mexico#cite_note-WRCC-6" target="_blank">Wikipedia</a> and a site called <a href="http://www.usclimatedata.com/climate/mimbres/new-mexico/united-states/usnm0517/2017/1" target="_blank">US Climate Data</a> The graph below shows the daily minimum and maximum temperatures for each month, as well as the total precipitation for each month.</p>
<p><img alt="location.jpg" src="own_figures/temp_online2.png" style="width:900px" /></p>
This table shows the climate of Silver City from 1901 to 1964
<p>&nbsp;</p>
<p><img alt="location.jpg" src="own_figures/temp_online1.png" style="width:500px" /></p>
And this picture shows the climate of Mimbres(Which is near the Silver City) from 1980 to 2010

<p><img alt="TMINTMAX.png" src="own_figures/TMINTMAX.png" style="height:300px; width:800px" /></p>
<p>We see that the min and max daily&nbsp;temperature almost agree with the ones we got from our data, once we translate Fahrenheit to Centigrade. There is only about 1~2 Centigrade difference.</p>

<p>&nbsp;<img alt="PRCP.png" src="own_figures/PRCP.png" style="height:333px; width:444px" /></p>
<p>To compare the precipitation&nbsp;we need to translate millimeter/day to inches/month. According to our analysis the average rainfall is 0.80 mm/day on Janurary which translates to about 24.0 mm/month on Janurary. According to US-Climate-Data the average rainfall is closer to 25 mm/month on Janurary. While in August, the average rainfall is 2.94 mm/day which equals 88.2 mm/month. Therefore, we can find that the our data is pretty close to the statistical data on Wikipedia and US Climate Data.

## 2. PCA analysis

For each of the six measurement, we compute the percentate of the variance explained as a function of the number of eigen-vectors used.

### 2.1. Percentage of variance explained.
![VarExplained1.png](own_figures/pca1.png)
We see that the top 5 eigen-vectors explain 45% of variance for TMIN, 58% for TOBS and 48% for TMAX.

We conclude that of the three, TOBS is best explained by the top 5 eigenvectors. This is especially true for the first eigen-vector which, by itself, explains 50% of the variance. But the first eigen-vector of TMIN and TMAX also performs good, which explains 35% and 39% respectively. It is safe to infer that the pattern of temperatrue is pretty clear and forms some rules so that the eigen-vectors are able to catch the main pattern hidden in the observed data.

![VarExplained2.png](own_figures/pca2.png)

The top 5 eigenvectors explain 8.2% of the variance for PRCP and 14% for SNOW. Both are low values. On the other hand the top 5 eigenvectors explain %74 of the variance for SNWD. This means that these top 5 eigenvectors capture most of the variation in the snow signals. Based on that we will dig deeper into the PCA analysis for snow-depth.

It makes sense that SNWD would be less noisy than SNOW. That is because SNWD is a decaying integral of SNOW and, as such, varies less between days and between the same date on diffferent years.

## 3. Analysis of Snow Depth
### 3.1. Eigen-Vector Interpretation
I choose to analyze the eigen-decomposition for snow-depth because the first 3 eigen-vectors explain 69% of the variance. 

Firstly, we graph the mean and the top 3 eigen-vectors. We observe that the snow season is from mid-November to the end of April, where the middle of February marks the peak of the snow-depth.
![SNWD_mean_eigs.png](own_figures/SNWDmeaneig.png)

Then we interpret the eigen-functions. The first eigen-function **eig1** has a shape very similar to the mean function. The main difference is that the eigen-function is a little bit close to zero during october-december while the mean is not.  The interpretation of this shape is that eig1 represents the overall amount of snow above/below the mean, but without changing the distribution over time. **eig2 and eig3** are similar in the following way. They all oscilate between positive and negative values. In other words, they correspond to changing the distribution of the snow depth over the winter months, but they don't change the total (much).

They can be interpreted as follows:
* **eig1:** overall amount of snow above/below the mean.
* **eig2:** start from Jan. to Apr., the snow depth keeps decreasing until mid March.
* **eig3:** more snow in late Dec., less snow in Feb., slightly more snow in March.

### 3.2. Reconstruction-based Analysis
In order to take a closer look about the eigen-vector so as to analyze the weather, I choose top-3 eigen-vectors to reconstuct the data records and compute its reconstruction residuals. For each data record, we can use  `decompose(row)` to extracts the series from the row, computes the residuals and constructs a new row that is reassembled into a dataframe. Note Coeff[i] here means the i^th coefficient of the corresponding (i-1)^th eigen-vector.
#### Coeff1
Coeff1: most positive
![SNWD_grid_Pos_coeff1.png](own_figures/recon_c1_pos.png)
Coeff1: most negative
![SNWD_grid_neg_coeff1.png](own_figures/recon_c1_neg.png)
Since we already knew that **eig1** represents the overall amount of snow above/below the mean. Therefore, large positive values of coeff1 correspond to more than average snow and low values correspond to less than average snow.

#### Coeff2
Coeff2: most positive
![SNWD_grid_Pos_coeff2.png](own_figures/recon_c2_pos.png)
Coeff2: most negative
![SNWD_grid_neg_coeff2.png](own_figures/recon_c2_neg.png)
Based on the meaning of **eig2** shown before, large positive values of coeff2 correspond to an early snow season. Negative values for coeff2 correspond to a late snow season. We can see in the picture, most of the snowfall is before mid feb, especially the very fist one picture, it snows mainly on Janurary. That's excatly explain that **eig2** shows the snow will decrease start from Jan and the depth of snow keeps decreasing. while in the second line, most of the snow is after mid-feb: the negative coeff2 is against the rule **eig2** shows.
#### Coeff3
Coeff3: most positive
![SNWD_grid_Pos_coeff3.png](own_figures/recon_c3_pos.png)
Coeff3: most negative
![SNWD_grid_neg_coeff3.png](own_figures/recon_c3_neg.png)

As **eig3** shows before, large positive values of coeff3 correspond to a snow season with two spikes: one in the late December, the other at the start of March. Negative values of coeff3 correspond to a season with a single peak around the Janurary. We can see that the picture shown above exactly represents these two situation, especially the left two, which have most significant inclination(either postive or negative) to **eig3**.

Besides the specific analysis of each coeffcient, we can also get a high level idea about these three eigen-vectors through the distribution of the corresponding coefficients as shown below.

<p><img alt="TMINTMAX.png" src="own_figures/recon_all_distribution.png" style="height:300px" /></p>

From this picture, we can easily find out how the data support each feature that eigen-vector represents. For the eig1, most corresponding coefficients are above zero, which means the snow depth follows the pattern of eigen-vector. The only difference is the level they supports. Comparing to eig1 which get supports from most of the data, eig2 and eig3 are kind of "controversial", especially eig3. Intuitively, based on our common knowledge, I would support eig2 more, because it is more likely to snow at Dec and Jan, and less likely to snow afterwards. And we can find that in the middle picture, although there might be some snow season reverse, but most of the coeffcients are positive and tend to agree with the opnion of the second eigen vector. As for the eig3, it represents the spike of the snow season, which contains more randomness, that's why we can see that the cumulative distribution is like the sigmoid function which means the supporters and opponents are half-half.  

### 3.3. Geographical Distribution Analysis
We can analyze the coeffcient distribution over elevation:
<p><img alt="TMINTMAX.png" src="own_figures/SNWD_ditribution_elevation.png" style="height:400px" /></p>

The distribution of SNWD over elevation also contains some information: at the low altitudes, the coefficients are almost the same. But when the elevation comes to 2200 or more, the coeff1 tends to become larger, which means the depth of snow becomes larger along with the elevation. Meanwhile, we can also find some outliers in the second and third graph, I think the main reason is when the elevation comes to around 3000, the snow depth varies a lot, because there may be some places hidden from sunshine so that the snow are not going to melt as fast as others. Which against the eig2. Of course, the usual area will speed up the melting process sine the temprature is going up, which will support the eig2. As for eig3, the randomness cames to become dominant, but the variation of coeff is also limits in 100. In conclusion, the eigen-vectors and the snow depth data interprets each other, and the depth is related to the elevation as we knew before. 

### 3.4. Temporal and Spatial Comparison
We now are going to estimate the relative importance of location-to-location variation relative to year-by-year variation. These are measured using the fraction by which the variance is reduced when we subtract from each station/year entry the average-per-year or the average-per-station respectively. Here are the results:

**coeff_1**<br>
total RMS                   =  557.468013648<br>
RMS removing mean-by-station=  396.027732492   Fraction explained: 28.9595595091<br>
RMS removing mean-by-year   =  455.207244626   Fraction explained: 18.3437913061<br>

**coeff_2** <br>
total RMS                   =  219.387409639<br>
RMS removing mean-by-station=  180.900661223   Fraction explained: 17.5428245766<br>
RMS removing mean-by-year   =  177.524794609   Fraction explained: 19.0815941074<br>

**coeff_3** <br>
total RMS                   =  167.950634625<br>
RMS removing mean-by-station=  165.161377532   Fraction explained: 1.66076007916<br>
RMS removing mean-by-year   =  136.586038427   Fraction explained: 18.6748899566<br>

We see that the variation by year and by station performs different on different eigen-vectors: for coeff_1, station explains more while for coeff_3, temporal factor came to be dominant. I think the main reason is that the first eigen vector have to deal with all the locations' snow depth, which makes it more sensitive to the location. While for eigen vector 3, it may want to convey the information about the year factor. That's why its coefficient explains much more fration.

## 4. Analysis of Precipitation
At first, let us have a look about the mean and its top-3 eigen-vectors via PCA:
<p><img alt="TMINTMAX.png" src="own_figures/PRCP_mean_eig.png" style="height:300px" /></p>
And the information(variance) explained by the eigen-vectors:
<p><img alt="TMINTMAX.png" src="own_figures/PRCP_eig_explain.png" style="height:300px" /></p>
Based on these two pictures, we can find there is a lot of noise in our precipitation data, which leads to that the PCA is not able to generate certain features, here represents as eigen-vectors, for the information hidden behind the precipitation. Like in the first picture, the only conclusion we may make is that precipitation varies a lot during July to October. And the second picture tells us that top eigen-vectors are useless because they only explains small percentage of the variance.

But there is another interesting thing we can work on: In our data, there are several stations around the monitoring area, maybe we can specify the precipitation for certain location and compute the relation between location and precipitation. As shown in the map below: the radius of each red circle represents the counts number of records, and the rectangle represents the are being monitored. Here we use the topographical map to get more information about the  topography. 
<p><img alt="TMINTMAX.png" src="own_figures/map2.png" style="height:270px" /></p>

However, it is likely to be hard to find correlations between the **amount** of rain on the same day in different stations. Because amounts of rain vary a lot between even close locations. It is more reasonable to try to compare whether or not it rained on the same day in different stations. As we see from the graph below: in our region it rains in about one ninth of the days.
<p><img alt="TMINTMAX.png" src="own_figures/PRCP_cum.png" style="height:320px" /></p>

Thus, we want to test for rejecting the null hypothesis: the rainfall in the two locations is independent. We restrict the data in which we have measurements for both stations, and compute the probability associated with the number of overlaps under the null hypothesis, which answers whether it rained in both locations.

### 4.1. Definition of the Statistical Test

Fix two stations, as stated before, we restrict our attention to the days for which we have measurements for both stations, and define the following notation:
* $m$ : the total number of days (for which we have measurements for both stations).
* $n_1$ : the number of days that it rained on station 1
* $n_2$ : the number of days that it rained on station 2
* $l$ : the number of days that it rained on both stations.

We want to calculate the probability P that the number of overlap days $l$ given $m,n_1,n_2$.

$$
P = {m \choose l,n_1-l,n_2-l,m-n_1-n_2+l} /{m \choose n_1}{m \choose n_2}
$$

Where
$$
{m \choose l,n_1-l,n_2-l,m-n_1-n_2+l} = \frac{m!}{l! (n_1-l)! (n_2-l)! (m-n_1-n_2+l)!}
$$

We use the fact that $\Gamma(n+1) = n!$ and denote $G(n) \doteq \log \Gamma(n+1)$
$$
\log P = \left[G(m) - G(l) -G(n_1-l) -G(n_2-l) -G(m-n_1-n_2+l) \right] - 
\left[G(m)-G(n_1)-G(m-n_1)\right] - \left[G(m)-G(n_2)-G(m-n_2)\right]
$$
Which slightly simplifies to 
$$
\log P = -G(l) -G(n_1-l) -G(n_2-l) -G(m-n_1-n_2+l) - G(m)+G(n_1)+G(m-n_1) +G(n_2)+G(m-n_2)
$$

The log probability scales with $m$ the length of the overlap. So to get a per-day significance we consider $
\frac{1}{m} \log P $

Based on the equation shown above, we can get the distribution of per-day significance of pair stations.

<p><img alt="TMINTMAX.png" src="own_figures/PRCP_sigdist2.png" style="height:320px" /></p>


### 4.2. Correlation Analysis
In order to analyze the precipitation relationship between station pairs, we can draw the significance map where the color represents the correlation.
<p><img alt="TMINTMAX.png" src="own_figures/PRCP_corrmap.png" style="height:400px" /></p>
We can find 3 groups of stations are highly correlated which are shown as deep green squares in the left top area. The most correlated group is the stations numbered from 10 to 28 (especially 10 to 20). And we can also find the station numbered from 40 to 50 is also highly correlated. To find whether geographical factor will influence the precipitation or not, we can plot the stations on its topographical map:
<p><img alt="TMINTMAX.png" src="own_figures/PRCP_geo.png" style="height:280px" /></p>

The black circle represents the No.10~No.20 stations that locates in higher places (around 1900 elevation) while the blue circle are the group of stations located in lower place (around 1400 elevation). It would be safe to conclude that the elevation can be a pretty important factor of precipitation.

### 4.3. PCA Analysis
<p><img alt="TMINTMAX.png" src="own_figures/PRCP_pca.png" style="height:280px" /></p>

The eigen-vectors performs pretty well from the picture shown above, which is to say, in terms of the probability that rains on pair stations, the relation forms certain pattern and can be expressed as certain features. 

<p><img alt="TMINTMAX.png" src="own_figures/PRCP_pca2.png" style="height:800px" /></p>

When we reorder the rows and columns of the matrix using one of the eigenvectors, we can find some hidden group of stations. For example, consider the bottom right corner of the scond matrix. The stations at positions 130-140 are clearly strongly correlated with each other. We can also find other similar groups or clusters by this block diagnal method.

### 4.4. Geographical Distribution of Coefficients
After we use the PCA, we plot the distribution of their corresponding coeffs. For every station, there are 4 triangles in different direction representing the top-4 eigen-vectors' coeffs. The size of the triangle represents the abs value of the coeff. And there are 2 kind of triangles: the triangle with filled color means the coeff is negative while the hollow triangle shows the positive value.
<p><img alt="TMINTMAX.png" src="own_figures/PRCP_geodist.png" style="height:320px" /></p>

The blue triangle represents the coeff_1 and the green one shows coeff_3 , which are almost all negative. Coeff_2(red) triangles and coeff_4(yellow) varies a lot on different location. It is really hard to make a conclusion about the relation between topography and the coefficients, but it is obvious within the valley, the coefficients tends to be larger to show the information about each eigen-vector. That is to say, the data collected from the valley area are more representative, and may have much more typical weather.
